# NumPy数组的计算： 通用函数
NumPy 数组的计算有时非常快，有时也非常慢。使 NumPy 变快的关键是利用向量化操作，通常在 NumPy 的通用函数（ufunc）中实现。本节将介绍 NumPy 通用函数的重要性——它可以提高数组元素的重复计算的效率；然后，将会介绍很多 NumPy 包中常用且有用的数学通用函数。
## 缓慢的循环
Python 的默认实现（被称作 CPython）处理起某些操作时非常慢，一部分原因是该语言的动态性和解释性——数据类型灵活的特性决定了序列操作不能像 C 语言和 Fortran 语言一样被编译成有效的机器码。

Python 的相对缓慢通常出现在很多小操作需要不断重复的时候，比如对数组的每个元素做循环操作时。假设有一个数组，我们想计算每个元素的倒数，一种直接的解决方法是：

In [2]:
import numpy as np
np.random.seed(0)
def compute_reciprocals(values):
    output=np.empty(len(values))
    for i in range (len(values)):
        output[i]=1.0/values[i]
    return output

values=np.random.randint(1,10,size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

这种实现方式可能对于有 C 语言或 Java 背景的人来说非常自然，但是如果测试一个很大量的输入数据运行上述代码的时间，这一操作将非常耗时，并且是超出意料的慢！我们将用 IPython 的 %timeit 魔法函数来测量：

In [3]:
big_array=np.random.randint(1,100,size=1000000)
%timeit compute_reciprocals(big_array)

2.25 s ± 160 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


完成百万次上述操作并存储结果花了几秒钟的时间！在手机都以 Giga-FLOPS（即每秒十亿次浮点运算）为单位计算处理速度时，上面的处理结果所花费的时间确实是不合时宜的慢。事实上，这里的处理瓶颈并不是运算本身，**而是 CPython 在每次循环时必须做数据类型的检查和函数的调度**。每次进行倒数运算时，Python 首先检查对象的类型，并且动态查找可以使用该数据类型的正确函数。如果我们在编译代码时进行这样的操作，那么就能在代码执行之前知晓类型的声明，结果的计算也会更加有效率。

## 通用函数介绍
NumPy 为很多类型的操作提供了非常方便的、静态类型的、可编译程序的接口，也被称作向量操作。你可以通过简单地对数组执行操作来实现，这里对数组的操作将会被用于数组中的每一个元素。这种向量方法被用于将循环推送至 NumPy 之下的编译层，这样会取得更快的执行效率。

比较以下两个结果：

In [4]:
%timeit compute_reciprocals(big_array)

2.28 s ± 66.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit (1.0 / big_array)

8.42 ms ± 679 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


计算一个较大数组的运行时间，可以看到它的完成时间比 Python 循环花费的时间要短。

NumPy 中的向量操作是通过通用函数实现的。通用函数的主要目的是对 NumPy 数组中的值执行更快的重复操作。它非常灵活，前面我们看过了标量和数组的运算，但是也可以对两个数组进行运算：

In [6]:
np.arange(5) / np.arange(1, 6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

通用函数并不仅限于一维数组的运算，它们也可以进行多维数组的运算：

In [7]:
x = np.arange(9).reshape((3, 3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]], dtype=int32)

## 探索NumPy的通用函数